#### Initialization ####

Contains ONLY functions, "default" package imports, and environment settings.

In [ ]:
print("Initializing project...")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### Package installer ####

Install a package if not already present.

In [ ]:
import sys
import subprocess
import pkg_resources

def install_if_needed(package):
    try:
        pkg_resources.get_distribution(package)
        print(f"{package} is already installed.")
    except pkg_resources.DistributionNotFound:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

#### Default packages ###

Load most-commonly used packages and install if not already present.

**Standard libraries**

In [ ]:
from io import StringIO
import csv
import os
import re
import requests

**Contrib libraries**

In [ ]:
install_if_needed('python-dotenv')
from dotenv import load_dotenv

install_if_needed('pandas')
import pandas as pd

install_if_needed('matplotlib')
import matplotlib.pyplot as plt

install_if_needed('networkx')
import networkx as nx

install_if_needed('numpy')
import numpy as np

install_if_needed('pydot')
import pydot

install_if_needed('pygraphviz')
import pygraphviz

#### Environment settings
Sensitive and machine-specific settings come from the "private" folder, whose files within it are NOT committed to the git repo for security reasons.

In [ ]:
def load_env_vars():
    load_dotenv('../private/.env')
    env_vars = {
        'CONGRESS_API_KEY': os.getenv('CONGRESS_API_KEY'),
        'CONGRESS_API_URL': os.getenv('CONGRESS_API_URL')
    }
    return env_vars

#### Helper functions ####

**Get variable name as string**

In [ ]:
def get_var_name(var):
    for varname, varvalue in globals().items():
        if varvalue is var:
            return varname

**Wrap long text labels**

In [ ]:
import textwrap
def split_label(label, max_length=25):
    return textwrap.fill(label, width=max_length)

**Save dataframe to csv**

In [ ]:
# Save df as csv based on environment.
def to_csv(df, filename=None, relpath='../../outputs', forcesave=False):
    if SAVE_DF == True | forcesave == True:
        if filename == None:
            filename = get_var_name(df)
    
        filepath = relpath + '/' + filename + '.csv'
        
        try:
            df.to_csv(filepath, index=False)
            print(f'DataFrame saved as CSV file in {filepath}')
        except IOError as e:
            print(f"An input/output error occurred: {e}")

**Show dataframe if desired**

In [ ]:
# Show dataframes based on environment.
def showdf(df, how='head', nrows=3, forceshow=False):
    if DISPLAY_DF == True | forceshow == True:
        if how == 'head':
            return df.head(nrows)
        elif how == 'tail':
            return df.tail(nrows)
        elif how == 'sample':
            return df.sample(nrows)

**Get congressional bill sponsors, cosponsors, and bill details**

In [ ]:
CONGRESS_API_KEY, CONGRESS_API_URL = load_env_vars()

def get_bill_cosponsors(bill_type, bill_number, congress):
    endpoint = f"bill/{congress}/{bill_type}/{bill_number}/cosponsors"
    params = {
        "api_key": CONGRESS_API_KEY
    }
    response = requests.get(CONGRESS_API_URL + endpoint, params=params)
    cosponsors = response.json()['cosponsors']
    df_cosponsors = pd.DataFrame(cosponsors, columns=['firstName', 'lastName', 'party'])
    df_cosponsors['type'] = 'cosponsor'
    df_cosponsors['bill'] = bill_type + bill_number
    return df_cosponsors

def get_bill_sponsors(bill_type, bill_number, congress):
    endpoint = f"bill/{congress}/{bill_type}/{bill_number}"
    params = {
        "api_key": CONGRESS_API_KEY
    }
    response = requests.get(CONGRESS_API_URL + endpoint, params=params)
    sponsors = response.json()['bill']['sponsors']
    df_sponsors = pd.DataFrame(sponsors, columns=['firstName', 'lastName', 'party'])
    df_sponsors['type'] = 'sponsor'
    df_sponsors['bill'] = bill_type + bill_number
    return df_sponsors

def get_bill_info(bill_type, bill_number, congress):
    df_cosponsors = get_bill_cosponsors(bill_type, bill_number, congress)
    df_sponsors = get_bill_sponsors(bill_type, bill_number, congress)
    df = pd.concat([df_cosponsors, df_sponsors])
    return df

#### Environment ####

In [ ]:
pd.options.display.max_columns = None # Do not truncate numerous columns.

#### Done ####

In [ ]:
print("...initialization complete.")